Use Case

As a physician, I want to calculate a patient's cardiovascular risk using the Framingham calculator based on LDL - http://reference.medscape.com/calculator/framingham-coronary-risk-ldl

Define Functional requirements

Step  - Define required clinical data elements
    Patient Age
    Patient Gender
    LDL Cholesterol in mg/dL
    HDL Cholesterol in mg/dL
    Blood Pressure
    Diabetes
    Smoking Status
    
Step  - Identify FHIR Resources to support use case data - http://hl7.org/fhir/resourcelist.html
    Patient Age - http://hl7.org/fhir/patient.html Patient.birthDate minus today's date
    Patient Gender - Patient.contact.gender
    LDL Cholesterol in mg/dL - Observation resource - http://www.hl7.org/fhir/diagnosticreport-example-lipids.html
    
    HDL Cholesterol in mg/dL - Observation resource
    Blood Pressure - Observation resource - http://hl7.org/fhir/observation-vitalsigns.html
        http://www.hl7.org/fhir/observation-example-bloodpressure.html
    Diabetes - Condition resource - http://hl7.org/fhir/condition.html
    Smoking Status - Observation resource - http://hl7.org/fhir/us/core/2017Jan/ValueSet-us-core-observation-ccdasmokingstatus.html
    

Step  - Identify data elements within each FHIR Resource definition necessary to support use case data


Patient Specific Data - FHIR Patient Resource

{
  "resourceType" : "Patient",
  // from Resource: id, meta, implicitRules, and language
  // from DomainResource: text, contained, extension, and modifierExtension
  "identifier" : [{ Identifier }], // An identifier for this patient
  "active" : <boolean>, // Whether this patient's record is in active use
  "name" : [{ HumanName }], // A name associated with the patient
  "telecom" : [{ ContactPoint }], // A contact detail for the individual
  "gender" : "<code>", // male | female | other | unknown
  "birthDate" : "<date>", // The date of birth for the individual
  // deceased[x]: Indicates if the individual is deceased or not. One of these 2:
  "deceasedBoolean" : <boolean>,
  "deceasedDateTime" : "<dateTime>",
  "address" : [{ Address }], // Addresses for the individual
  "maritalStatus" : { CodeableConcept }, // Marital (civil) status of a patient
  // multipleBirth[x]: Whether patient is part of a multiple birth. One of these 2:
  "multipleBirthBoolean" : <boolean>,
  "multipleBirthInteger" : <integer>,
  "photo" : [{ Attachment }], // Image of the patient
  "contact" : [{ // A contact party (e.g. guardian, partner, friend) for the patient
    "relationship" : [{ CodeableConcept }], // The kind of relationship
    "name" : { HumanName }, // A name associated with the contact person
    "telecom" : [{ ContactPoint }], // A contact detail for the person
    "address" : { Address }, // Address for the contact person
    "gender" : "<code>", // male | female | other | unknown
    "organization" : { Reference(Organization) }, // C? Organization that is associated with the contact
    "period" : { Period } // The period during which this contact person or organization is valid to be contacted relating to this patient
  }],
  "animal" : { // This patient is known to be an animal (non-human)
    "species" : { CodeableConcept }, // R!  E.g. Dog, Cow
    "breed" : { CodeableConcept }, // E.g. Poodle, Angus
    "genderStatus" : { CodeableConcept } // E.g. Neutered, Intact
  },
  "communication" : [{ // A list of Languages which may be used to communicate with the patient about his or her health
    "language" : { CodeableConcept }, // R!  The language which can be used to communicate with the patient about his or her health
    "preferred" : <boolean> // Language preference indicator
  }],
  "generalPractitioner" : [{ Reference(Organization|Practitioner) }], // Patient's nominated primary care provider
  "managingOrganization" : { Reference(Organization) }, // Organization that is the custodian of the patient record
  "link" : [{ // Link to another patient resource that concerns the same actual person
    "other" : { Reference(Patient|RelatedPerson) }, // R!  The other patient or related person resource that the link refers to
    "type" : "<code>" // R!  replaced-by | replaces | refer | seealso - type of link
  }]
}

In [3]:
from fhirclient import client
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta

# Define the FHIR Endpoint
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://fhirtest.uhn.ca/baseDstu3'
}
smart = client.FHIRClient(settings=settings)

# Perform a GET on patient with id of 'cf-1495289345197'
import fhirclient.models.patient as p
patient = p.Patient.read('cf-1497294721585', smart.server)

# Print the first (i.e. [0] element in the patient.name
print ('Patient\'s Name = ',patient.name[0].given[0],patient.name[0].family)

# Define variable DOB as the patient's date of birth in a string format
DOB = patient.birthDate.isostring

# Print patient's gender
gender = patient.gender
print ('Gender = ',gender)

# Print the patient's date of birth
print ('DOB = ', DOB)

# Define and print today variable
now = datetime.datetime.today()

print ('Today\'s Date = ', now)

#Define DOB2 date object
DOB2 = datetime.datetime.strptime(DOB, '%Y-%m-%d')

# Calcuate patient's age using rdelta method of the dateutil module
rdelta = relativedelta(now, DOB2)
age = rdelta.years
# print the years from the rdelta object
print ('Patient\'s age = ',age)

Patient's Name =  Joseph Framingham
Gender =  male
DOB =  1953-02-14
Today's Date =  2017-07-05 16:04:31.273386
Patient's age =  64


In [4]:
#LDL
import fhirclient.models.observation as obs
search = obs.Observation.where(struct={'patient':"Patient/cf-1497294721585",'code':"18262-6"})
LDL = search.perform_resources(smart.server)
if LDL:
    LDL_val = LDL[0].valueQuantity.value
    print(str(LDL_val) + " " + LDL[0].valueQuantity.unit)

212 mg/dL


In [5]:
# Perform a GET on Observation with id of 'LabHDL'

search = obs.Observation.where(struct={'patient':"Patient/cf-1497294721585",'code':"2085-9"})
HDL = search.perform_resources(smart.server)
if HDL:
    HDL_val = HDL[0].valueQuantity.value
    print(str(HDL_val) + " " + HDL[0].valueQuantity.unit)

70 mg/dL


In [6]:
observation = obs.Observation.read("c135db1b-42e1-4e82-98a6-768848c7f088", smart.server)

search = obs.Observation.where(struct={'patient':"Patient/cf-1497294721585",'code':"55284-4"})
press = search.perform_resources(smart.server)
if press:
    sys = press[0].component[0].valueQuantity.value
    dia = press[0].component[1].valueQuantity.value
    print("Systolic: "+str(sys)+" "+press[0].component[0].valueQuantity.unit)
    print("Diastolic: "+str(dia)+" "+press[0].component[0].valueQuantity.unit)

Systolic: 128 mmHg
Diastolic: 86 mmHg


In [1]:
import json
import requests

response = requests.get("https://fhirtest.uhn.ca/baseDstu3/ValueSet/DiabetesExamples/$expand")
json_data = json.loads(response.text)
expansion = json_data["expansion"]
code_list = expansion["contains"]
#Include the parent snomed code
parent = json_data["compose"]["include"][0]["filter"][0]["value"]
code_list[0]["code"]
parent
json_data



{'compose': {'include': [{'filter': [{'op': 'is-a',
      'property': 'concept',
      'value': '73211009'}],
    'system': 'http://snomed.info/sct'}]},
 'expansion': {'contains': [{'code': '609569007',
    'display': 'Diabetes mellitus due to genetic defect in insulin action (disorder)',
    'system': 'http://snomed.info/sct'},
   {'code': '46635009',
    'display': 'Diabetes mellitus type 1 (disorder)',
    'system': 'http://snomed.info/sct'},
   {'code': '44054006',
    'display': 'Diabetes mellitus type 2 (disorder)',
    'system': 'http://snomed.info/sct'},
   {'code': '8801005',
    'display': 'Secondary diabetes mellitus (disorder)',
    'system': 'http://snomed.info/sct'},
   {'code': '111552007',
    'display': 'Diabetes mellitus without complication (disorder)',
    'system': 'http://snomed.info/sct'},
   {'code': '703136005',
    'display': 'Diabetes mellitus in remission',
    'system': 'http://snomed.info/sct'},
   {'code': '609568004',
    'display': 'Diabetes mellitus du

In [8]:
diab_codes = []
for x in range(len(code_list)):
    diab_codes.append(code_list[x]["code"])
diab_codes.append(parent)

In [9]:
import fhirclient.models.condition as cond

diab_flag = False
for x in diab_codes:
        search = cond.Condition.where(struct={'patient':'Patient/cf-1497294721585','code':x})
        type1 = search.perform_resources(smart.server)
        if type1:
            diab_flag = True
    
#condition that says whether they have one of these two types of diabetes
if diab_flag:
    diabetes = True
    print("They have diabetes")

They have diabetes


In [10]:
import fhirclient.models.observation as obs
search = obs.Observation.where(struct={'patient':'Patient/cf-1497294721585','code':'72166-2'})
smoking = search.perform_resources(smart.server)
if smoking:
    smoke = True
    print("They smoke")


They smoke


Coronary Risk Calculator

In [11]:
points = 0

if gender == "male":
    if age <= 34:
        points += -1
    if age >= 40 and age <= 44:
        points += 1
    if age >= 45 and age <= 49:
        points += 2
    if age >= 50 and age <= 54:
        points += 3
    if age >= 55 and age <= 59:
        points += 4
    if age >= 60 and age <= 64:
        points += 5
    if age >= 65 and age <= 69:
        points += 6
    if age >= 70:
        points += 7
    if LDL_val < 100:
        points += -3
    if LDL_val >= 160 and LDL_val <= 189:
        points += 1
    if LDL_val >= 190:
        points += 2
    if HDL_val < 35:
        points += 2
    if HDL_val >= 35 and LDL_val <= 44:
        points += 1
    if HDL_val >= 60:
        points += -1
    if sys <=129 and dia >= 85 and dia <= 89:
        points += 1
    if sys >= 130 and sys <=139 and dia < 90:
        points += 1
    if sys <= 139 and dia >= 90 and dia <= 99:
        points += 2
    if sys >= 140 and sys <=159 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 2
    if smoking == True:
        points += 2

In [12]:

if gender == "female":
    if age <= 34:
        points += -9
    if age >= 35 and age <= 39:
        points += -4
    if age >= 45 and age <= 49:
        points += 3
    if age >= 50 and age <= 54:
        points += 6
    if age >= 55 and age <= 59:
        points += 7
    if age >= 60:
        points += 8
    if LDL_val < 100:
        points += -2
    if LDL_val >= 160:
        points += 2
    if HDL_val < 35:
        points += 5
    if HDL_val >= 35 and HDL_val <= 44:
        points += 2
    if HDL_val >= 45 and HDL_val <= 49:
        points += 1
    if HDL_val >= 60:
        points += -2
    if sys < 120 and dia < 80:
        points += -3
    if sys <=159 and dia >=90 and dia < 100:
        points += 2
    if sys > 159 or dia > 99:
        points += 3
    if diabetes == True:
        points += 4
    if smoking == True:
        points += 2

In [13]:
male_results = {0:3,1:4,2:4,3:6,4:7,5:9,6:11,7:14,8:18,9:22,10:27,11:33,12:40,13:47}
female_results = {0:2,1:2,2:3,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:11,11:13,12:15,13:17,14:20,15:24,16:27}

if points >=14 and gender == "male":
    risk = ">= 56%"
elif points >=17 and gender == "female":
    risk = ">= 32%"
elif gender == "male":
    percent = male_results[points]
    risk = str(percent)+"%"
else:
    percent = female_results[points]
    risk = str(percent)+"%"
print("Total number of points: "+str(points))
print("Risk of Heart Disease: "+risk)

Total number of points: 9
Risk of Heart Disease: 22%
